<a href="https://colab.research.google.com/github/vlady98ish/Colab/blob/main/HW_2_1.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports



In [97]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt

In [98]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


#Init Array of data for future graph

In [99]:
mean_epocs = []
num_of_hidden_neuron = [1,2,4]
bridge = [True,False]
std_epocs = []
learning_rate = [0.1,0.01]

#Init Linear Layer Class

In [100]:
class Linear(torch.nn.Module):
  def __init__(self, in_features: int, out_features: int, bias: bool = True, device=None, dtype=None) -> None:
    factory_kwargs = {'device': device, 'dtype': dtype}
    super(Linear, self).__init__()
    self.in_features = in_features
    self.out_features = out_features
    self.weight = nn.Parameter(torch.empty((out_features, in_features), **factory_kwargs))
    if bias:
        self.bias = nn.Parameter(torch.empty(out_features, **factory_kwargs))
    else:
        self.register_parameter('bias', None)
    self.reset_parameters()

  def reset_parameters(self) -> None:
    self.weight = nn.Parameter(torch.rand([self.out_features, self.in_features]))
    if self.bias is not None:
      self.bias = nn.Parameter(torch.rand([self.out_features]))

  def forward(self, input: torch.Tensor) -> torch.Tensor:
    return torch.matmul(input, torch.transpose(self.weight,0,1)) + self.bias

  def extra_repr(self) -> str:
    return 'in_features={}, out_features={}, bias={}'.format(
        self.in_features, self.out_features, self.bias is not None
      )

#Init BTU Class

In [101]:
class BTU(torch.nn.Module):
  def __init__(self, T=0.2, inplace: bool = False):
      super(BTU, self).__init__()
      self.T = T

  def forward(self, input: torch.Tensor) -> torch.Tensor:
      return 1 / (1 + torch.exp(-input/self.T))

#Init XOR Model

In [102]:
dim = 2
out_dim = 1


x_train = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], requires_grad=True, dtype=torch.float32)
print("X_Train tensor:",x_train)
t_train = torch.tensor([[0], [1], [1], [0]], dtype=torch.float32)
print("T_Train tensor:",t_train)
###MY CODE#####
x_validation = torch.tensor([[0, 0],[0, 1],[1, 0],[1, 1],[1,0.1],[1,0.9],[0.9,0.9],[0.1,0.9]], requires_grad=True, dtype=torch.float32)
print("X_Validation tensor:",x_validation)
t_validation = torch.tensor([[0], [1], [1], [0],[1],[0],[0],[1]])
print("T_Validation tensor:",t_validation)
###MY CODE#####

class XOR_Net_Model(nn.Module):
  def __init__(self,num_hidden, bypass=True):
    super().__init__()
    self.bypass = bypass
    self.hidden = Linear(dim, num_hidden)
    if self.bypass:
      self.output = Linear(num_hidden + dim, out_dim)
    else:
      self.output = Linear(num_hidden, out_dim)
    self.BTU = BTU(0.5)

  def forward(self, input):
    z1 = self.hidden(input)
    y1 = self.BTU(z1)
    if self.bypass:
      y1_concat = torch.cat((input, y1), 1)
      z2 = self.output(y1_concat)
    else:
      z2 = self.output(y1)
    return self.BTU(z2)

X_Train tensor: tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]], requires_grad=True)
T_Train tensor: tensor([[0.],
        [1.],
        [1.],
        [0.]])
X_Validation tensor: tensor([[0.0000, 0.0000],
        [0.0000, 1.0000],
        [1.0000, 0.0000],
        [1.0000, 1.0000],
        [1.0000, 0.1000],
        [1.0000, 0.9000],
        [0.9000, 0.9000],
        [0.1000, 0.9000]], requires_grad=True)
T_Validation tensor: tensor([[0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [1]])


#Init Loss Function

In [103]:
def Loss(out, t_train):
  return -torch.sum(t_train * torch.log(out) + (1.0 - t_train) * torch.log(1.0 - out))  # Cross Entropy loss function

#Train Function

In [104]:
def train(model, x_train, t_train, optimizer):
  y_pred = model(x_train)
  loss = Loss(y_pred, t_train)

  # zero gradients berfore running the backward pass
  optimizer.zero_grad()

  # backward pass to compute the gradient of loss
  # backprop + accumulate 
  loss.backward()

  # update params
  optimizer.step()
  return loss

#Test Function

In [105]:
# define test step operation:
def test(model, x_test, t_test):
  loss = Loss(model(x_test), t_test)
  return loss

#Print results of experiment

In [106]:
def print_results(epocs_arr,validation_loss_arr,train_loss_arr, bad_training_count):
  mean_of_epocs = torch.mean(torch.FloatTensor(epocs_arr))
  std_of_epocs = torch.std(torch.FloatTensor(epocs_arr)) 
  mean_validation = torch.mean(torch.FloatTensor(validation_loss_arr))
  std_validation = torch.std(torch.FloatTensor(validation_loss_arr)) 
  mean_loss = torch.mean(torch.FloatTensor(train_loss_arr))
  std_loss = torch.std(torch.FloatTensor(train_loss_arr)) 



  mean_epocs.append(mean_of_epocs)
  std_epocs.append(std_of_epocs)
  print("-----------------------------------------------------\n")
  print(f"Mean of epocs = {mean_of_epocs} \nStandard deviation of epocs = {std_of_epocs}\n")
  print(f"Mean of validation loss = {mean_validation} \nStandard deviation of validation loss = {std_validation}\n")
  print(f"Mean of train  loss = {mean_loss} \nStandard deviation of train loss = {std_loss}\n")
  print(f"Bad training count = {bad_training_count}")


#MODEL RUN FUNCTION
10 runs

In [107]:
def model_run10(learning_rate,hidden,bypass,epocs):
  good_training_count =0
  bad_training_count = 0
  epoc_count = []
  validation_loss_data = []
  training_loss_data = []
  parameters_of_good_training =[]
  while(good_training_count!=10):
    model = XOR_Net_Model(hidden, bypass)
    
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    counter_of_10 = 0
    previous_loss = 1000000
 
  
    for i in range(epocs):
      train_loss = train(model, x_train, t_train, optimizer)
      ###MY CODE#####
      validation_loss = test(model,x_validation,t_validation)
      difference = previous_loss - validation_loss
      # print("Diffetence of loss function: {}".format(difference))
      if difference <= 0.0001 and validation_loss<0.2:
        counter_of_10+=1
        print("COUNTER {}".format(counter_of_10))
      else:
        counter_of_10 = 0
      if counter_of_10 == 10:
        print("Good train")
        print("Validation Loss: {} ,Train Loss: {}, Count of epocs: {}".format(validation_loss,train_loss,i))

        epoc_count.append(i)
        validation_loss_data.append(validation_loss)
        training_loss_data.append(train_loss)
        good_training_count+=1

        break
      elif i == 40000:
        print("Bad train: number of epocs = {}".format(i))
        bad_training_count+=1
        break  
      previous_loss = validation_loss
      ###MY CODE#####
      
      # print("\n Epoc: %s, train loss: %s" % (i, train_loss))
      # print("\n Epoc: %s, validation loss: %s" % (i, validation_loss))
      # print(model(x_train))
  print_results(epoc_count,validation_loss_data,training_loss_data, bad_training_count)
  print("--------------------------------------------------\n")
  print("Hyper Parameters")
  print(f"Learning Rate = {learning_rate}\n Num of hidden neurons = {hidden} \n Bypass = {bypass} \n Max epocs ={epocs}")

#Training

#<b>First Test</b><br>
1-Test with Learning rate = 0.1<br>
2- Count of hidden neuron = 2<br>
3 - Bypass = True


In [108]:
l_rate = 0.1
num_hidden = 2
bypass = True
num_epocs = 400000

In [109]:
model_run10(l_rate,num_hidden,bypass,num_epocs)

COUNTER 1
COUNTER 2
COUNTER 3
COUNTER 4
COUNTER 5
COUNTER 6
COUNTER 7
COUNTER 8
COUNTER 9
COUNTER 10
Good train
Validation Loss: 0.17036321759223938 ,Train Loss: 0.03723892197012901, Count of epocs: 1744
COUNTER 1
COUNTER 1
COUNTER 2
COUNTER 3
COUNTER 4
COUNTER 5
COUNTER 6
COUNTER 7
COUNTER 8
COUNTER 9
COUNTER 10
Good train
Validation Loss: 0.17085173726081848 ,Train Loss: 0.0347508043050766, Count of epocs: 1484
COUNTER 1
COUNTER 2
COUNTER 3
COUNTER 4
COUNTER 5
COUNTER 6
COUNTER 7
COUNTER 8
COUNTER 9
COUNTER 10
Good train
Validation Loss: 0.11366834491491318 ,Train Loss: 0.03242792561650276, Count of epocs: 1201
COUNTER 1
COUNTER 2
COUNTER 3
COUNTER 4
COUNTER 5
COUNTER 6
COUNTER 7
COUNTER 8
COUNTER 9
COUNTER 10
Good train
Validation Loss: 0.14514198899269104 ,Train Loss: 0.03177657350897789, Count of epocs: 1646
COUNTER 1
COUNTER 1
COUNTER 2
COUNTER 3
COUNTER 4
COUNTER 5
COUNTER 6
COUNTER 7
COUNTER 8
COUNTER 9
COUNTER 10
Good train
Validation Loss: 0.17791995406150818 ,Train Loss: 0.0

#<b>Second Test</b><br>
1-Test with Learning rate = 0.1<br>
2- Count of hidden neuron = 2<br>
3 - Bypass = False

In [110]:
l_rate = 0.1
num_hidden = 2
bypass = False

In [111]:
model_run10(l_rate,num_hidden,bypass,num_epocs)

Bad train: number of epocs = 40000
Bad train: number of epocs = 40000
COUNTER 1
COUNTER 2
COUNTER 3
COUNTER 4
COUNTER 5
COUNTER 6
COUNTER 7
COUNTER 8
COUNTER 9
COUNTER 10
Good train
Validation Loss: 0.08503921329975128 ,Train Loss: 0.02879149094223976, Count of epocs: 1102
COUNTER 1
COUNTER 2
COUNTER 3
COUNTER 4
COUNTER 5
COUNTER 6
COUNTER 7
COUNTER 8
COUNTER 9
COUNTER 10
Good train
Validation Loss: 0.08509735763072968 ,Train Loss: 0.028512727469205856, Count of epocs: 1192
Bad train: number of epocs = 40000
COUNTER 1
COUNTER 2
COUNTER 3
COUNTER 4
COUNTER 5
COUNTER 6
COUNTER 7
COUNTER 8
COUNTER 9
COUNTER 10
Good train
Validation Loss: 0.08496096730232239 ,Train Loss: 0.028720930218696594, Count of epocs: 1186
COUNTER 1
COUNTER 2
COUNTER 3
COUNTER 4
COUNTER 5
COUNTER 6
COUNTER 7
COUNTER 8
COUNTER 9
COUNTER 10
Good train
Validation Loss: 0.07943274080753326 ,Train Loss: 0.02998104691505432, Count of epocs: 1024
Bad train: number of epocs = 40000
Bad train: number of epocs = 40000
COUNTER

#<b>Third Test</b><br>
1-Test with Learning rate = 0.1<br>
2- Count of hidden neuron = 4<br>
3 - Bypass = True

In [112]:
l_rate = 0.1
num_hidden = 4
bypass = True

In [113]:
model_run10(l_rate,num_hidden,bypass,num_epocs)

COUNTER 1
COUNTER 2
COUNTER 3
COUNTER 4
COUNTER 5
COUNTER 6
COUNTER 7
COUNTER 8
COUNTER 9
COUNTER 10
Good train
Validation Loss: 0.09237433969974518 ,Train Loss: 0.023483101278543472, Count of epocs: 1087
COUNTER 1
COUNTER 2
COUNTER 3
COUNTER 4
COUNTER 5
COUNTER 6
COUNTER 7
COUNTER 8
COUNTER 9
COUNTER 10
Good train
Validation Loss: 0.09409839659929276 ,Train Loss: 0.024755673483014107, Count of epocs: 1133
COUNTER 1
COUNTER 2
COUNTER 3
COUNTER 4
COUNTER 5
COUNTER 6
COUNTER 7
COUNTER 8
COUNTER 9
COUNTER 10
Good train
Validation Loss: 0.11305362731218338 ,Train Loss: 0.02820698544383049, Count of epocs: 1146
COUNTER 1
COUNTER 1
COUNTER 2
COUNTER 3
COUNTER 4
COUNTER 5
COUNTER 6
COUNTER 7
COUNTER 8
COUNTER 9
COUNTER 10
Good train
Validation Loss: 0.09886597096920013 ,Train Loss: 0.028669165447354317, Count of epocs: 1202
COUNTER 1
COUNTER 2
COUNTER 3
COUNTER 4
COUNTER 5
COUNTER 6
COUNTER 7
COUNTER 8
COUNTER 9
COUNTER 10
Good train
Validation Loss: 0.14008702337741852 ,Train Loss: 0.0311643

#<b>Fourth Test</b><br>
1-Test with Learning rate = 0.1<br>
2- Count of hidden neuron = 4<br>
3 - Bypass = False

In [114]:
l_rate = 0.1
num_hidden = 4
bypass = False

In [115]:
model_run10(l_rate,num_hidden,bypass,num_epocs)

COUNTER 1
COUNTER 2
COUNTER 3
COUNTER 4
COUNTER 5
COUNTER 6
COUNTER 7
COUNTER 8
COUNTER 9
COUNTER 10
Good train
Validation Loss: 0.09070996940135956 ,Train Loss: 0.030979514122009277, Count of epocs: 1131
COUNTER 1
COUNTER 2
COUNTER 3
COUNTER 4
COUNTER 5
COUNTER 6
COUNTER 7
COUNTER 8
COUNTER 9
COUNTER 10
Good train
Validation Loss: 0.08149021863937378 ,Train Loss: 0.02677975594997406, Count of epocs: 998
COUNTER 1
COUNTER 2
COUNTER 3
COUNTER 4
COUNTER 5
COUNTER 6
COUNTER 7
COUNTER 8
COUNTER 9
COUNTER 10
Good train
Validation Loss: 0.09080256521701813 ,Train Loss: 0.030547931790351868, Count of epocs: 1129
COUNTER 1
COUNTER 2
COUNTER 3
COUNTER 4
COUNTER 5
COUNTER 6
COUNTER 7
COUNTER 8
COUNTER 9
COUNTER 10
Good train
Validation Loss: 0.08373968303203583 ,Train Loss: 0.028176307678222656, Count of epocs: 1327
COUNTER 1
COUNTER 2
COUNTER 3
COUNTER 4
COUNTER 5
COUNTER 6
COUNTER 7
COUNTER 8
COUNTER 9
COUNTER 10
Good train
Validation Loss: 0.09033917635679245 ,Train Loss: 0.03147423639893532,

#<b>Fifth Test</b><br>
1-Test with Learning rate = 0.01<br>
2- Count of hidden neuron = 2<br>
3 - Bypass = True

In [116]:
l_rate = 0.01
num_hidden = 2
bypass = True

In [ ]:
model_run10(l_rate,num_hidden,bypass,num_epocs)

COUNTER 1
COUNTER 2
COUNTER 3
COUNTER 4
COUNTER 5
COUNTER 6
COUNTER 7
COUNTER 8
COUNTER 9
COUNTER 10
Good train
Validation Loss: 0.19986987113952637 ,Train Loss: 0.043300800025463104, Count of epocs: 13661
COUNTER 1
COUNTER 2
COUNTER 3
COUNTER 4
COUNTER 5
COUNTER 6
COUNTER 7
COUNTER 8
COUNTER 9
COUNTER 10
Good train
Validation Loss: 0.19985055923461914 ,Train Loss: 0.04693758487701416, Count of epocs: 13302
COUNTER 1
COUNTER 2
COUNTER 3
COUNTER 4
COUNTER 5
COUNTER 6
COUNTER 7
COUNTER 8
COUNTER 9
COUNTER 10
Good train
Validation Loss: 0.19987131655216217 ,Train Loss: 0.04288490116596222, Count of epocs: 13136
COUNTER 1
COUNTER 2
COUNTER 3
COUNTER 4
COUNTER 5
COUNTER 6
COUNTER 7
COUNTER 8
COUNTER 9
COUNTER 10
Good train
Validation Loss: 0.19988791644573212 ,Train Loss: 0.04550010710954666, Count of epocs: 16008


#<b>Six Test</b><br>
1-Test with Learning rate = 0.01<br>
2- Count of hidden neuron = 2<br>
3 - Bypass = False

In [ ]:
l_rate = 0.01
num_hidden = 2
bypass = False

In [ ]:
model_run10(l_rate,num_hidden,bypass,num_epocs)

#<b>Seven Test</b><br>
1-Test with Learning rate = 0.01<br>
2- Count of hidden neuron = 4<br>
3 - Bypass = False

In [ ]:
l_rate = 0.01
num_hidden = 4
bypass = False

In [ ]:
model_run10(l_rate,num_hidden,bypass,num_epocs)

#<b>Eight Test</b><br>
1-Test with Learning rate = 0.01<br>
2- Count of hidden neuron = 4<br>
3 - Bypass = True

In [ ]:
l_rate = 0.01
num_hidden = 4
bypass = True

In [ ]:
model_run10(l_rate,num_hidden,bypass,num_epocs)
  

#<b>Nine Test</b><br>
1-Test with Learning rate = 0.1<br>
2- Count of hidden neuron = 1<br>
3 - Bypass = True

In [ ]:
l_rate = 0.1
num_hidden = 1
bypass = True

In [ ]:
model_run10(l_rate,num_hidden,bypass,num_epocs)

#Plot function

In [ ]:
def plot_function(x,y,x_name,y_name):
  plt.plot(x, y)
  plt.title(f"{x_name} and {y_name}")
  plt.xlabel(x_name)
  plt.ylabel(y_name)
  plt.show


#Plot graphs

In [ ]:
#First graph 
plot_function(mean_epocs,num_of_hidden_neuron, "Mean of Epocs", "Number of hidden neurons")
#Second graph  
plot_function(mean_epocs,bridge, "Mean of Epocs", "Bridge")
#Third graph
plot_function(std_epocs,learning_rate, "Mean of Epocs", "Learning Rate")

